<img src="AW&H2015.png" style="float: left">

<img src="flopylogo.png" style="float: center">

# Modifying PEST:  Adding an additional parameter to our cross sectional model

The goal of model calibration is to find a model that has the best history match __while also__ using reasonable parameters.  In our manual trial-and-error and our PEST setup of the cross sectional model we had only one parameter hk1 = horizontal hydraulic conductivity for the aquifer. But, our cross-sectional model is simple - recall that it looked like this:

<img src="xsect_figure.png" style="float: center">

So perhaps one parameter is all we need for a good forecast. Or perhaps not - how should we think of this?  In *Applied Groundwater Modeling (2nd edition)* by Anderson et al. (2015) we see that uncertainty in a model's forecast is directly related to the amount of simplicity/complexity that is brought to bear: 

<img src="Fig10.2_MML_curves.png" style="float: center">

You can see that our model can be __too complex__ and make the model forecast more uncertain.  But less recognized, our model can also be __too simple__ and we can handcuff the model's ability to simulate the forecast thus in effect increasing uncertainty in the forecast. This tradeoff has been called the "Goldilocks Principle", or in economics, the "law of Diminishing Returns", which looks something like this:


<img src="Hunt1998_sweetspot.png" style="float: center">


Unfortunately too much complexity or oversimplification is not easy to recognize *a priori*.  During this class we'll look at both cases to help you identify the "sweet spot" for reduced uncertainty.

In this notebook we won't perform the calculations as shown in figure 10.2. Rather, we'll take an easier "proof-is-in-the-pudding" approach:  we'll add another parameter to our cross-section model and see how much the forecast uncertainty changes. 

Why do it this way?  Because PEST++ has made it very easy to track forecast uncertainty for you, the approach we use here is the easiest way to evaluate model oversimplification.  And it focuses your attention on the __*forecast*__ which is why most models are built in the first place.  And it is straightforward as it only consists of:

 1. Creating a quick and simple model, estimate uncertainty in your forecasts
 2. Adding parameter(s), then evaluating changes in forecast uncertainties.
 
 

 ### Objectives:  
 
 1) Add a new parameter to an existing PEST run by making a new template file
 
 2) Update the PEST control file so it sees and knows what to do with the new parameter
 
 3) See how forecast uncertainty can change as model complexity changes
 
 4) Emphasize that what we do here is a great first step to better, more certain, model forecasts!

# Getting back into the xsec cross-sectional model

In the xsec_pest_setup exercise you noted the forecast uncertainty for h02_8, which is the head in the 8th node from the left calculated during the second stress period. Let's add another parameter and see what happens to the uncertainty in this forecast.

## Recall that the profile model looks like ths:

<img src="xsect_figure.png" style="float: center">

We'll make the flux boundary condition on the right side (the green node) a parameter.  By taking a parameter that is "fixed" (= not changed during parameter estimation) and making it adjustable, we increase the model complexity.  Again, first we'll do some Python notebook prep (push shift-enter in the next code block)


In [ ]:
import shutil, os

### First copy over the raw model files;  for this activity, we are providing the files you need.  This next block of code copies them to your */acitivities/xsec_k_and_flux* subdirectory for you.


In [ ]:
base_dir = os.path.join("..","..","models","10par_xsec","raw_model_files")
[shutil.copy2(os.path.join(base_dir,f),f) for f in os.listdir(base_dir)]

# 1) Now we'll create another *template (.TPL)* file

We are going to make the rightmost flux a parameter.  But, recall that our problem had __two__ stress periods, an initial calibration period followed by a forecast period:

<img src="xsect_problem_fig1.png" style="float: center">

So, there are __two__ boundary condition flux values, one for each stress period, that we need to make parameters. Why should we add a parameter for the second (future/forecast) stress period?

The easiest way to make a template file is to modify an existing input file. A flux boundary condition is entered into MODFLOW using the WEL Package input file.  Also recall these steps from our xsec_pest_setup notebook: 

 1. Look inside __xsect_k_flux.pst__ to see which forward model is to be run. 
 2. It's MODFLOW, so read the NAM file to find the name of the WEL Package input file
 3. Open the .wel file in a text editor
 4. Make a copy of the input file and name it the same as the input file but with .tpl extension
 5. Add a new line on top of your tpl file to tell PEST that it is a template file and what the delimiter is
 6. Substitute the variable __sp1_flux__ and __sp2_flux__ surrounded by the delimiter you chose where appropriate
 7. Save the file and run TEMPCHEK 

#### Reminder of the Rules for constructing TPL Files 

 1. The first line of the TPL file must identify that it is a template file by listing "`ptf ~`" where "`~`" is a "parameter delimiter" that tells PEST where a parameter sits in the file. We used a tilde here but it can be any symbol. __However__, whatever delimiter symbol is listed in the first line must be used consistently throughout that template file.
 2. The template file looks like the original model input file __BUT__ parameters are substituted for the  model input(s) that we want PEST to estimate.  Parameters are identified by surrounding the parameter name listed in the PEST control (.pst) file with the delimiter.  

## How does PEST know that we created another Template file?

### We have to __modify__ the PEST Control File.
The PEST control file (.pst) is the main driver of the PEST run. It has most all of the important information needed to direct PEST, so it is a logical place to start. Recall that the top of the file has these PEST variables taken from Appendix 1 from SIR 2010-5169 we handed out:

<img src="2010-5169_Appendix1_PST_file.png" style="float: center">

 
 We have to modify the number of parameters (NPAR) because we are adding one (bcflux).  And because a flux parameter is different than hydraulic conductivity, we should add another parameter group (NPARGP).  
 
 1. Open __xsect.pst__ in a text editor. 
 2. Increase NPAR by 2 (as we are adding two parameters)
 3. Increase NPARGP by 1 (because both parameters are the same type)
 4. Increase NTPLFILE by 1 (both parameters are in one new template file)
 
We also have to give the information for this new parameter.  This is in the 'parameter groups' ( * parameter group) and 'parameter data' (* parameter data) sections further down in the PEST control file  (__Hint:__  The PEST input is more easily naviagated by searching for the * that demarcates a new PEST input section). 
 
 <img src="2010-5169_Appendix1_PST_file_parameter.png" style="float: center">
 
 
4. Make a new blank line below the existing line in * parameter groups
5. Copy the line above into the new blank line
6. Change the first entry in the line PARGPNME to a new parameter group name - let's use __fluxbc__

now let's do something similar to add the parameter data:

7. Make a new blank line below the existing line in * parameter data
8. Copy the line above into the new blank line
9. Change the first entry PARNME to match what you entered for a parameter name in your template file (__sp1_flux__)
10. Change the initial value PARVAL1 to the value that was in the WEL input file (0.5)
11. Change PARLBND to 0.0001 (this is the lower bound that PEST will use)
12. Change PARUBND to 10000 (this is the upper bound that PEST will use)
13. Change PARGP to match the parameter group name __fluxbc__
14. Make a new blank line below this new line, copy the line we just created into the blank space
15. In this second new line change (__sp1_flux__) to (__sp2_flux__)
16. For __sp2_flux__, change PARVAL1 from 0.5 to 1.0 to match what was in the original .wel file

Moreover, we have to tell PEST what is the name of the new template file, and what filename it should use when it creates the model input file.  That is done in the "* model input/output section:

<img src="2010-5169_Appendix1_PST_file_input_output.png" style="float: center">

so we need to:

14. Add a blank line after the first line in the section that has the existing tpl file from the xsec_pest_setup exercise.
15. Add the template file name (TEMPFLE variable), a space, then the model input file (INFILE variable) that we want PEST to create when it uses the template

But we are not done yet with the PEST control file.  We've made a lot of changes to the file, and a small change to PEST setup, so we should do another NOPTMAX=0 run to test the plumbing.  

16. Check if NOPTMAX in the 9th line of the control file = 0
17. Add our forecast from last activity (add __++forecasts(h02_8)__ to the very bottom of the file)
17. Save the PEST control file as xsec_k_flux.pst

Whew - that is a lot of things to manipulate for just one additional template file!  But this flexibility is needed once we move to less trivial models.  And one can now really see the need for PESTCHEK! So,finally (!) run PESTCHEK at the command line by typing:  

__pestchek xsec_k_flux.pst__  

or 

__./pestchek xsec_k_flux.pst__ 


for Windows or Mac, respectively. 

### Digression:  "Oh no - PESTCHEK didn't like my PEST control file!"

If you got an error don't take it personally, happens all the time (that is why John Doherty wrote PESTCHEK in the first place). Rather, take a deep breath and look at the PESTCHEK report: you'll see that PESTCHEK is very good at guiding your eyes to what needs fixing. Give it a go if you got an error - if you can't figure it out grab one of the instructors.  

# 2) Now that we have our new template file and told PEST about it, let's run PEST++!

When error free we can now focus on the PESTCHEK warning that the control file has `NOPTMAX=0`, which is what we want because the model only is run once, and then PEST++ processes all the output and reports the objective function phi. This checks that we have all the information flowing out of and back into PEST.  And, if we don't like the objective function distribution we can reweight, then re-run PEST++ with `NOPTMAX=0` again. 

### Run PEST++ on xsec_k_flux.pst 

__pest++ xsec_k_flux.pst__ (Windows) or __./pestpp xsec_k_flux.pst__ (Mac)


Look at the `.rec` file to see results. See where your new template file information is echoed?


### Change NOPTMAX, re-run PEST++ on xsec_k_flux.pst

Once we see that everything is plumbed as we had intended, open __xsec_k_flux.pst__ in a text editor and change NOPTMAX to 20, rerun PEST++. Look at the `.rec` file again to see results. 

#### Search for "FINAL OPTIMISATION RESULTS" in your rec file


# What happened to our Calibration?

Not much, our measurement objective function Phi is about the same good fit.


# More importantly, what happened to our forecast uncertainty?   
Was there a change in forecast uncertainty from increasing the number of parameters to 3?  Was the uncertainty larger or smaller?  

# Let's see if we can get back to where we were in xsect_pest_setup

You can turn a parameter on and off, where "on" = adjustable and "off" = fixed or not adjustable.  Change  your __sp1_flux__  and __sp2_flux__ parameter from "log" to "fixed" in the PEST control file.  Save the file, rerun PESTCHEK and PEST++.  

Did you get the same uncertainty for the forecast that you saved from the xsect_pest_setup activity?

## Now this next bit is important!
PEST++ has made this forecast analysis very easy on you:

__It used your lower and upper bounds to set the uncertainty around the parameters__ as it expects you view those bounds as a reasonable range of the parameter values.  But, we had you make the range very large.  Let's narrow the range to something more reasonable.  

1) Open __xsect_k_flux.pst__ in a text editor,  and make all three parameter adjustable again

2) change PARLBND to 0.1 and PARUBND to 2.0 for both __sp1_flux__ and __sp2_flux__, and save the file  

3) Run PESTCHEK 

4) if no error rerun PEST++ on the file, and look at the change in the forecast uncertainty reported in the rec file.

__Aside:__ When we estimate the uncertainty around our parameters before calibration it is called "building our prior". You'll hear a lot more about that this week.  

### What is the change in parameter uncertainty when the parameter bounds were tightened?  


### And more to the point, was the forecast uncertainty for H02_8 larger or smaller?  



## Now, let's make sure we know how uncertainty is reported:

1) Open up your PEST control file __'xsect_k_flux.pst'__, find the "parameter data" section, and change the word "none" to "log".

##### This log transforms the parameters, which is recommended for any parameter that should not go negative (like hydraulic conductivity). 

2) Run pestchek and PEST++ again

3) Open up your __xsect_k_flux.rec__ file again and look at the bottom of the file.

#### Hmm, the fit (as shown by the reported Phi) looks similar. Why is the prior mean of the parameter sp2_flux = 0!!  

Is the forecast log or non-log tranformed?

Did the forecast H02_8 and its uncertainty change when we log transformed the parameters?

# Hmm, did any of these changes effect what was reported for uncertainty around sp2_flux?  Why or why not?

# Advanced - playing with fixing and freeing parameters and its effect on uncertainty 

Now explore the combinations of fixing and freeing  parameters. Rename and save the rec file after each run for comparison:

1) __hk1__ and __sp1_flux__ adjustable; __sp2_flux__ fixed


*where did sp2_flux go on the bottom of the rec file?*


2) __sp1_flux__ adjustable; __hk1__ and __sp2_flux__ fixed

3) __sp1_flux__ and __sp2_flux__ adjustable; __hk1__ fixed

and most interestingly:

4) __sp2_flux__ adjustable; __hk1__ and __sp1_flux__ fixed

**********

### Was the reported uncertainty larger when more parameters are included as we expect from the theory as shown in Figure 10.2 at the top of this notebook? 

### What is going on with combination 4 above?

### Add the starting head at the left boundary a parameter as a parameter, change its bounds, and evaluate the change in forecast uncertainty.

### One last parting thought:  We added one additional type of parameter, what *other* simplifications does our model have that may lead it to be oversimplified?